<a href="https://colab.research.google.com/github/Sky0307/2022DLProject/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import pandas as pd
import numpy as np
import h5py
from torch.utils.data import Dataset, DataLoader

In [ ]:
# link to download the zip format of the data
download_data_url = "https://drive.google.com/file/u/1/d/1X3amA5n6RjYoY5QHdFfYOFfh9w3zbEU4/view"

# link to the actual dataset
data_url = "https://drive.google.com/drive/folders/1OUdNIIV1SD7eavLNWC8a3mpxqnGjL1Kb?usp=sharing"

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
directory = "drive/MyDrive/datasets"

# Default Params

In [19]:
class Parameters:
    def __init__(self):
        self.batch_size = None
        self.label_size = 10
        self.epoch_num = None
        self.sample_rate = 16000
        self.use_cuda = True
        self.cuda = torch.cuda.is_available() and self.use_cuda
        self.log_step = None
        self.dataset_len = None
        self.dictionary = {0: 'pop',
                           1: 'metal',
                           2: 'disco',
                           3: 'blues',
                           4: 'reggae',
                           5: 'classical',
                           6: 'rock',
                           7: 'hiphop',
                           8: 'country',
                           9: 'jazz'}
        self.r_dictionary = {'pop': 0,
                             'metal': 1,
                             'disco': 2,
                             'blues': 3,
                             'reggae': 4,
                             'classical': 5,
                             'rock': 6,
                             'hiphop': 7,
                             'country': 8,
                             'jazz': 9}

        # On pure GTZAN dataset
        self.TRAIN_DATA_PATH = directory + '/train.h5'
        self.VAL_DATA_PATH = directory + '/valid.h5'
        self.TEST_DATA_PATH = directory + '/test.h5'

        # On hybrid dataset
        self.A_TRAIN_DATA_PATH = directory + '/fin_train.h5'
        self.A_VAL_DATA_PATH = directory + '/fin_valid.h5'
        self.A_TEST_DATA_PATH = directory + '/fin_test.h5'

        # On hybrid dataset
        self.LA_TRAIN_DATA_PATH = directory + '/l_train.h5'
        self.LA_VAL_DATA_PATH = directory + '/l_valid.h5'
        self.LA_TEST_DATA_PATH = directory + '/l_test.h5'

        self.MODEL_SAVE_FOlD = '../model/'
        self.LOG_SAVE_FOLD = '../log/'

        if self.cuda:
            self.kwargs = {'num_workers': 1, 'pin_memory': True}
        else:
            self.kwargs = {}

        self.learning_rate = 1e-5

    def __str__(self):
        out_string = "The Batch Size is {0}\n" \
                     "The Label Size is {1}\n" \
                     "The Epoch Num is {2}\n" \
                     "The Cuda is set to {3}\n" \
                     "The log step is {4}".format(self.batch_size,
                                                  self.label_size,
                                                  self.epoch_num,
                                                  self.cuda,
                                                  self.log_step)
        return out_string

In [20]:
class TorchData(Dataset):

    def __init__(self, dataset_path):
        """
        Take the h5py dataset
        """
        super(TorchData, self).__init__()
        self.dataset = h5py.File(dataset_path, 'r')
        self.mel = self.dataset['mel']
        self.tag = self.dataset['tag']

        self.len = self.mel.shape[0]

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        mel = self.mel[index].astype(np.float32)
        mel = np.reshape(mel, (1, mel.shape[0], mel.shape[1]))
        mel = torch.from_numpy(mel)
        tag = torch.from_numpy(self.tag[index].astype(np.float32))
        sample = {"mel": mel, "tag": tag}

        return sample


# define the data loaders
def torch_dataset_loader(dataset, batch_size, shuffle, kwargs):
    """
    take the h5py dataset
    """
    loader = DataLoader(TorchData(dataset),
                        batch_size=batch_size,
                        shuffle=shuffle,
                        **kwargs)
    return loader

In [21]:
Para = Parameters()
print(Para)

train_loader = torch_dataset_loader(Para.TRAIN_DATA_PATH, Para.batch_size, True, Para.kwargs)
validation_loader = torch_dataset_loader(Para.VAL_DATA_PATH, Para.batch_size, False, Para.kwargs)
test_loader = torch_dataset_loader(Para.TEST_DATA_PATH, Para.batch_size, False, Para.kwargs)

fin_train_loader = torch_dataset_loader(Para.A_TRAIN_DATA_PATH, Para.batch_size, True, Para.kwargs)
fin_validation_loader = torch_dataset_loader(Para.A_VAL_DATA_PATH, Para.batch_size, False, Para.kwargs)
fin_test_loader = torch_dataset_loader(Para.A_TEST_DATA_PATH, Para.batch_size, False, Para.kwargs)

l_train_loader = torch_dataset_loader(Para.LA_TRAIN_DATA_PATH, Para.batch_size, True, Para.kwargs)
l_validation_loader = torch_dataset_loader(Para.LA_VAL_DATA_PATH, Para.batch_size, False, Para.kwargs)
l_test_loader = torch_dataset_loader(Para.LA_TEST_DATA_PATH, Para.batch_size, False, Para.kwargs)

for index, data_item in enumerate(fin_train_loader):
    print(data_item['mel'].shape)
    print(data_item['tag'].shape)
    break

The Batch Size is None
The Label Size is 10
The Epoch Num is None
The Cuda is set to True
The log step is None
torch.Size([1, 128, 128])
torch.Size([10])
